In [1]:
# Load the path from Valhalla

import requests
import json

def get_distance(pointA, pointB):
    url = "http://valhalla:8002/route"
    data =  {
        "locations":[
            {
                "lat":pointA[1],
                "lon":pointA[0],
                "type":"break"
            },{
                "lat":pointB[1],
                "lon":pointB[0],
                "type":"break"
            }
        ],
        "costing":"pedestrian",
        "directions_options":{
            "units":"miles"
        }
    }

    data_json = json.dumps(data)
    response = requests.post(url, data=data_json)
    response_obj = json.loads(response.text)
    #distance = response_obj['trip']['summary']['length']
    time_in_seconds = response_obj['trip']['summary']['time']
    #west = response_obj['trip']['summary']['min_lon']
    #south = response_obj['trip']['summary']['min_lat']
    #east = response_obj['trip']['summary']['max_lon']
    #north = response_obj['trip']['summary']['max_lat']
    #geometry = decode_geometry(response_obj['trip']['legs'][0]['shape'])
    return time_in_seconds/60

In [2]:
import psycopg2
conn = psycopg2.connect(host="postgres",database="osm", user="osm", password="osm")
cursor = conn.cursor()

lat = 40.4952385
lon = -74.4439958
distance_in_meters = 5000
amenity = 'restaurant'

cursor.execute("""
select * FROM (
    select
     name,
     ST_X(ST_Transform(way, 4326)) as lon,
     ST_Y(ST_Transform(way, 4326)) as lat,
     ST_DISTANCE(
         way,
         ST_TRANSFORM(
             ST_SETSRID(
                 ST_GeomFromText(
                     'POINT(%f %f)'
                 ),
             4326),
         3857)
     ) AS distance
    FROM
       planet_osm_point
    WHERE
      amenity = '%s'
) AS subquery WHERE distance < %f 
ORDER BY distance;
""" % (lon, lat, amenity, distance_in_meters))
pointA = [lon, lat]
returned_data = []
for record in cursor:
    pointB = [record[1], record[2]]
    time = get_distance(pointA, pointB)
    print (record[0], "(%f min)" % time, record[3])
    returned_data.append(record + (time,))
    
cursor.close()


indochine (0.150000 min) 31.074602482557
Teri Teri Japanese Restaurant (0.300000 min) 39.1919317367822
Garden State Ale House (0.600000 min) 69.2063645738383
Hotoke (0.833333 min) 93.3933521156638
Panico's Brick Oven Pizza (1.316667 min) 93.7376430640566
My Way Korean Restaurant (0.833333 min) 95.0082891699802
Shalimar Halal Gyro Grill (0.900000 min) 101.42240974455
Salt Seafood & Oyster Bar (1.433333 min) 133.905433785702
Ramen Nagomi (2.066667 min) 148.212932556758
Steakhouse 85 (1.766667 min) 157.394200222384
Fiesta Mexican Restaurant (1.450000 min) 160.10961500766
Filippo's Famous Pizza (1.766667 min) 195.478279451293
Old Man Rafferty's (2.300000 min) 196.342101257956
Catherine Lombardi Restaurant (1.933333 min) 213.216244381457
Stage Left (2.333333 min) 222.9482753767
Due Mari (3.533333 min) 269.520397611065
INC American Bar and Kitchen (3.516667 min) 389.40160405114
KBG Korean BBQ & Grill (5.116667 min) 421.491258359564
RU Hungry (10.783333 min) 728.559538999156
honeygrow (10.916

In [6]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Bring the returned data into a pandas dataframe
df = pd.DataFrame(returned_data, columns = ["Name", "Lon", "Lat", "Distance", "Time"])

# zip the lon and lat into a coordinates column
df['Coordinates'] = list(zip(df.Lon, df.Lat))

# Now we can get rid of those lon and lat columns
df.drop('Lon', axis=1, inplace=True)
df.drop('Lat', axis=1, inplace=True)

# GeoPandas needs a Shapely geometry, so convert that lon,lat to a Point
df['Coordinates'] = df['Coordinates'].apply(Point)

gdf = gpd.GeoDataFrame(df, geometry='Coordinates')
print (gdf)


                                     Name     Distance       Time  \
0                               indochine    31.074602   0.150000   
1           Teri Teri Japanese Restaurant    39.191932   0.300000   
2                  Garden State Ale House    69.206365   0.600000   
3                                  Hotoke    93.393352   0.833333   
4               Panico's Brick Oven Pizza    93.737643   1.316667   
5                My Way Korean Restaurant    95.008289   0.833333   
6               Shalimar Halal Gyro Grill   101.422410   0.900000   
7               Salt Seafood & Oyster Bar   133.905434   1.433333   
8                            Ramen Nagomi   148.212933   2.066667   
9                           Steakhouse 85   157.394200   1.766667   
10              Fiesta Mexican Restaurant   160.109615   1.450000   
11                 Filippo's Famous Pizza   195.478279   1.766667   
12                     Old Man Rafferty's   196.342101   2.300000   
13          Catherine Lombardi Res

In [57]:
# Draw Path on a leaflet map

from ipyleaflet import Map, basemaps, basemap_to_tiles, GeoJSON, Popup
from ipywidgets import HTML
from shapely.geometry import shape, MultiPoint
import json
import math

# This isn't a ipyleaflet function for some reason!
def fitBounds(m, geojson):
    # Center the map on our shape!
    # Draw a buffer around the whole thing
    polygon = MultiPoint(list(map(lambda x: x['geometry']['coordinates'],geojson['features']))).convex_hull
    new_bounds = polygon.buffer(0.001).bounds
    new_center = [(new_bounds[1] + new_bounds[3]) / 2, (new_bounds[0] + new_bounds[2]) / 2]
    new_zoom = math.floor(
            min(math.log(180/(new_bounds[2] - new_bounds[0]), 2), math.log(360/(new_bounds[3] - new_bounds[1]), 2)))

    m.center = new_center
    m.zoom = new_zoom
    return m

m = Map(
    layers=(basemap_to_tiles(basemaps.CartoDB.DarkMatter), )
)

# There's got to be a better way to do popups!
out_data = json.loads(gdf.to_json())
popups = list(map(lambda x: Popup(child=HTML(value=x['properties']['Name']),
                                  location=(x['geometry']['coordinates'][1],
                                            x['geometry']['coordinates'][0])),
                                  out_data['features']))

def click_handler(event=None, id=None, properties=None, type=None, coordinates=None):
    if (id):
        m.add_layer(popups[int(id)])
        
out_data = json.loads(gdf.to_json())
geo_json = GeoJSON(data=out_data, onEachLayer=(lambda x: print (x)))
geo_json.on_click(click_handler)

m.add_layer(geo_json)

m = fitBounds(m, out_data)


m

Map(basemap={'url': 'https://{s}.tile.openstreetmap.org/{z}/{x}/{y}.png', 'max_zoom': 19, 'attribution': 'Map …